In [1]:
# enabling 3rd party widgets
# from google.colab import output
# output.enable_custom_widget_manager()
# output.disable_custom_widget_manager()

# interactive 3D plot
# !pip install ipympl
# %matplotlib widget

In [2]:
import os
import math
import numpy as np
import matplotlib.pyplot as plt

import time as time
import platform as platform

import h5py

In [3]:
colab_flag = False

strategy = None
# strategy = tf.distribute.MirroredStrategy()

In [4]:
current_sys = platform.system()

if current_sys == 'Windows':
    dir_sep = '\\'
else:
    dir_sep = '/'

In [5]:
if colab_flag == True:
    from google.colab import drive
    drive.mount('/content/drive')
    os.chdir('/content/drive/MyDrive/Github/MLROM/KS/')

In [6]:
print(os.getcwd())

/home/rkaushik/Documents/Thesis/MLROM/new_cdv


In [7]:
from tools.misc_tools import create_CDV_data, compute_lyapunov_spectrum

In [8]:
# setting seed for PRNGs
prng_seed = 42
np.random.seed(prng_seed)

FTYPE = np.float64
ITYPE = np.int64

# Lorenz System

In [9]:
# setting up params (and saving, if applicable)
params_mat = np.array([
    [0.95, -0.76095, 0.1, 1.25, 0.2, 0.5],
#     [0.99, -0.801*0.99, 0.1, 1.25, 0.2, 0.5]
])
print(params_mat)
num_cases = params_mat.shape[0]

t0 = 0.0
T = 51000.0
delta_t = 0.1

return_params_arr = False
normalize_flag = False
alldata_withparams_flag = False

init_state = np.array([0.9, 0.2, -0.1, -0.5, -0.2, 0.2], dtype=FTYPE)

[[ 0.95    -0.76095  0.1      1.25     0.2      0.5    ]]


### Creating initial data, for transient inspection and cutoff

In [10]:
time_creation = time.time()
res_dict = create_CDV_data(
    T, t0, delta_t, 
    params_mat, init_state,
    return_params_arr=return_params_arr,
    normalize=normalize_flag,
    alldata_withparams=alldata_withparams_flag, FTYPE=FTYPE, ITYPE=ITYPE
)
time_creation = time.time() - time_creation
print('time_creation : {:.0f} s'.format(time_creation))

all_data = res_dict['all_data']
N = res_dict['N']
boundary_idx_arr = res_dict['boundary_idx_arr']
params_arr = res_dict['params_arr']
normalization_constant_arr = res_dict['normalization_constant_arr']

time_creation : 134 s


In [11]:
# making relevant helper arrays to cut off initial transient
initial_t0 = 1000
T -= initial_t0


initial_t0 = np.array(
    [initial_t0]*params_mat.shape[0], dtype=FTYPE) # try to keep all entries the same
initial_idx = np.zeros_like(initial_t0, dtype=ITYPE)

begin_idx = 0
len_relevant_idx = 0
for i in range(len(initial_t0)):
    init_idx = ITYPE( (initial_t0[i] + 0.5*delta_t)//delta_t )
    initial_idx[i] = init_idx + begin_idx
    len_relevant_idx += boundary_idx_arr[i] - initial_idx[i]
    begin_idx = boundary_idx_arr[i]

relevant_idx = np.empty(shape=len_relevant_idx, dtype=ITYPE)
start = 0
for i in range(len(initial_t0)):
    end = start + (boundary_idx_arr[i] - initial_idx[i])
    relevant_idx[start:end] = np.arange(initial_idx[i], boundary_idx_arr[i])
    start = end

In [12]:
print('boundary_idx_arr :', boundary_idx_arr)
print('initial_idx :', initial_idx)

boundary_idx_arr : [510001]
initial_idx : [10000]


In [13]:
# cutting off initial transient
all_data = all_data[relevant_idx]
# init_idx = 0
# for i in range(len(boundary_idx_arr)):
#     init_idx += (initial_t0[i] + 0.25*delta_t)//delta_t
#     boundary_idx_arr[i] -= init_idx

prev_boundary_idx = 0
for i in range(num_cases):
    boundary_idx_arr[i] = prev_boundary_idx + (boundary_idx_arr[i] - initial_idx[i])
    prev_boundary_idx = boundary_idx_arr[i]

In [14]:
boundary_idx_arr

array([500001])

### Saving the data

In [15]:
# making ae save directory
dir_name_data = os.getcwd() + dir_sep + 'saved_data'
if not os.path.isdir(dir_name_data):
    os.makedirs(dir_name_data)

counter = 0
while True:
    dir_check = 'data_' + str(counter).zfill(3)
    if os.path.isdir(dir_name_data + dir_sep + dir_check):
        counter += 1
    else:
        break

dir_name_data = dir_name_data + dir_sep + dir_check
os.makedirs(dir_name_data)
print('dir_name_data:', dir_name_data)


# saving sim data
sim_data = {
    'params_mat':params_mat,
    't0':t0,
    'T':T,
    'delta_t':delta_t,
    'return_params_arr':return_params_arr,
    'normalize_flag':normalize_flag,
    'alldata_withparams_flag':alldata_withparams_flag,
    'prng_seed':prng_seed,
}
with open(dir_name_data+dir_sep+'sim_data_params.txt', 'w') as f:
    f.write(str(sim_data))

dir_name_data: /home/rkaushik/Documents/Thesis/MLROM/new_cdv/saved_data/data_004


In [16]:
# saving the data
np.savez(
    dir_name_data+dir_sep+'data',
    all_data=all_data,
    boundary_idx_arr=boundary_idx_arr,
    normalization_constant_arr=[normalization_constant_arr],
    initial_t0=initial_t0,
    init_state_org=init_state,
    # init_state_mat=init_state_mat,
    # lyapunov_spectrum_mat=lpspectrum_mat,
    )

### Computing the Lyapunov spectrum

In [17]:
cdf_kwargs = {
    'T':10000,
    't0':0,
    'delta_t':delta_t,
    'return_params_arr':False,
    'normalize':False,
    'alldata_withparams':False,
    'FTYPE':np.float64,
    'ITYPE':np.int64
}

create_data_fn = create_CDV_data
num_modes = 6

init_state_mat = np.empty(shape=(num_cases, num_modes), dtype=FTYPE)
begin_idx = 0
for i in range(num_cases):
    init_state_mat[i, :] = all_data[begin_idx, 0:num_modes]
    begin_idx = boundary_idx_arr[i]

dy = 1e-9
dy_mat = np.empty(shape=num_cases, dtype=FTYPE)
begin_idx = 0
for i in range(num_cases):
    # dy_mat[i] = dy * np.mean(  np.linalg.norm( all_data[begin_idx:boundary_idx_arr[i]], axis=1 )/xgrid.shape[0]  ) timeMean of spaceMeanofSpaceNorm
    # dy_mat[i] = dy * np.mean( np.linalg.norm( all_data[begin_idx:boundary_idx_arr[i], 0:num_modes], axis=0 )/( (boundary_idx_arr[i]-begin_idx)**0.5 ) ) # spaceMean of timeRMS
    dy_mat[i] = np.mean( np.mean(all_data[begin_idx:boundary_idx_arr[i], 0:num_modes]**2, axis=0)**0.5 )
    # print(dy_mat[i])
    dy_mat[i] *= dy
    begin_idx = boundary_idx_arr[i]
    print('dy_mat[{}] :'.format(i), dy_mat[i])
print('')

# cleaning up
del(all_data)
del(boundary_idx_arr)
del(params_arr)
del(normalization_constant_arr)
del(relevant_idx)
del(res_dict)

lpspectrum_mat, _ = compute_lyapunov_spectrum(
    create_data_fn, cdf_kwargs, num_modes, 
    init_state_mat, params_mat, dy_mat,
    zeta=10, delta_completionratio=0.1, num_exp=None, print_flag=True)

dy_mat[0] : 3.7248332512042945e-10

number of evaluation intervals per case: 10000

case 1 completion_ratio : 0.1, elapsed_time : 19.177s, global_completion : 10.0%
case 1 completion_ratio : 0.2, elapsed_time : 37.942s, global_completion : 20.0%
case 1 completion_ratio : 0.3, elapsed_time : 56.877s, global_completion : 30.0%
case 1 completion_ratio : 0.4, elapsed_time : 76.252s, global_completion : 40.0%
case 1 completion_ratio : 0.5, elapsed_time : 95.375s, global_completion : 50.0%
case 1 completion_ratio : 0.6, elapsed_time : 114.877s, global_completion : 60.0%
case 1 completion_ratio : 0.7, elapsed_time : 134.298s, global_completion : 70.0%
case 1 completion_ratio : 0.8, elapsed_time : 153.546s, global_completion : 80.0%
case 1 completion_ratio : 0.9, elapsed_time : 171.811s, global_completion : 90.0%
case 1 completion_ratio : 1.0, elapsed_time : 190.24s, global_completion : 100.0%
case 1 MLE : 0.027087



In [18]:
lpspectrum_mat.sort(axis=1)
lpspectrum_mat = lpspectrum_mat[:, -1::-1]

In [19]:
print(lpspectrum_mat[:, 0])

[0.02708692]


In [20]:
def find_DKY(lp_spectrum):
    # lp_spectrum is sorted largest to smallest
    running_sum = np.empty_like(lp_spectrum)
    running_sum[:] = lp_spectrum
    for i in range(1, lp_spectrum.shape[0]):
        running_sum[i] += running_sum[i-1]
    idx = np.where(running_sum >= 0)[0]
    idx.sort()
    idx = idx[-1]
    DKY = (idx+1) + running_sum[idx]/np.abs(lp_spectrum[idx+1])
    return DKY

In [21]:
DKY_lst = []
for i in range(lpspectrum_mat.shape[0]):
    DKY = find_DKY(lpspectrum_mat[i])
    print(DKY, lpspectrum_mat[i])
    DKY_lst.append(DKY)

2.321165029661567 [ 0.02708692 -0.00164554 -0.07921591 -0.10164063 -0.21914134 -0.22544346]


### Computing new data based on 'initial states' from the transient cutoffs and total time from mean Lyapunov time

In [22]:
with np.load(dir_name_data+dir_sep+'data.npz', allow_pickle=True) as fl:
    all_data = np.array(fl['all_data'])
    boundary_idx_arr = np.array(fl['boundary_idx_arr'])
    normalization_constant_arr = np.array(fl['normalization_constant_arr'])
    initial_t0 = np.array(fl['initial_t0'])
    init_state_org = np.array(fl['init_state_org'])

In [23]:
# saving the data
np.savez(
    dir_name_data+dir_sep+'data',
    all_data=all_data,
    boundary_idx_arr=boundary_idx_arr,
    normalization_constant_arr=normalization_constant_arr,
    initial_t0=initial_t0,
    init_state_org=init_state_org,
    init_state_mat=init_state_mat,
    lyapunov_spectrum_mat=lpspectrum_mat,
    DKY_lst=np.array(DKY_lst)
)

In [24]:
lp_dict = {
    'lp_exp':list(lpspectrum_mat),
    'DKY_lst':list(DKY_lst),
}
with open(dir_name_data+dir_sep+'lp_exp.txt', 'w') as f:
    f.write(str(lp_dict))

In [25]:
# fl = np.load(dir_name_data+dir_sep+'data.npz', allow_pickle=True)

In [26]:
# fl.files